In [399]:
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
import math, random

In [400]:
def xor(*args):
    assert len(args) >= 2 and min(map(len, args)) == max(map(len, args))
    if len(args) == 2:
        a,b = args
        return bytes([x^y for (x, y) in zip(a,b)])
    else:
        return xor(args[0], xor(*args[1:]))
    
def key_xor(msg, key):
    stream = key * (len(msg) // len(key)) + key[:(len(msg) % len(key))]
    return xor(msg, stream)

def pad(msg, block_size):
    extra_bytes = block_size - len(msg) % block_size 
    return msg + bytes([extra_bytes] * extra_bytes)

class PaddingException(Exception):
    pass

def unpad(msg):
    if not msg or not 0 < msg[-1] <= 16 or msg[-msg[-1]:] != bytes([msg[-1]] * msg[-1]):
        raise PaddingException
    return msg[:-msg[-1]]

def random_bytes(len):
    return bytes([random.randint(0,255) for _ in range(len)])

In [56]:
def ctr_keystream(nonce, count, cipher):
    return cipher.encrypt(nonce + count.to_bytes(8, byteorder='little'))

def ctr(msg, nonce, cipher):
    out = b''
    count = 0
    for i in range(len(msg) // 16):
        out += xor(ctr_keystream(nonce, count, cipher), msg[i*16:(i+1)*16])
        count += 1
    out += xor(ctr_keystream(nonce, count, cipher)[:len(msg)%16], msg[len(msg)-len(msg)%16:])
    return out

### Challenge 25

In [57]:
key = random_bytes(16)
nonce = random_bytes(8)
cipher = AES.new(key, AES.MODE_ECB)

In [58]:
ecb_cipher = AES.new(b'YELLOW SUBMARINE', AES.MODE_ECB)

with open('25.txt', 'r') as file:
    ciphtxt = ctr(ecb_cipher.decrypt(b64decode(''.join(file.readlines()))), nonce, cipher)

In [49]:
def edit(ciphtxt, key, offset, newtxt):
    output = bytearray(ciphtxt)
    output[offset*16:(offset+1)*16] = xor(ctr_keystream(nonce, offset, cipher), newtxt)
    return bytes(output)

In [40]:
keystream = bytes([])

for i in range(len(ciphtxt) // 16):
    keystream += edit(ciphtxt, key, i, bytes(16))[i*16:(i+1)*16]
    
plaintxt = xor(keystream, ciphtxt)
print(plaintxt.decode('ascii'))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


### Challenge 26

In [59]:
def enc1(msg):
    msg = msg.replace(';', '%3b').replace('=', '%3d')
    msg = "comment1=cooking%20MCs;userdata=" + msg + ";comment2=%20like%20a%20pound%20of%20bacon"
    return ctr(msg.encode('ascii'), nonce, cipher)

def dec1(ciphtxt):
    return ";admin=true;" in ctr(ciphtxt, nonce, cipher).decode('ascii', errors='replace')

In [60]:
ciphtxt = bytearray(enc1('00000xadminxtrue'))
ciphtxt[32:48] = xor(ciphtxt[32:48], b'00000x00000x0000', b'00000;00000=0000')
print(ctr(ciphtxt, nonce, cipher).decode('ascii', errors='replace'))
dec1(bytes(ciphtxt))

comment1=cooking%20MCs;userdata=00000;admin=true;comment2=%20like%20a%20pound%20of%20bacon


True

### Challenge 27

In [84]:
def cbc_encrypt(msg, IV, cipher):
    out = bytes([])
    curr_block = IV[:]
    for i in range(len(msg) // 16):
        curr_block = xor(curr_block, msg[i*16:(i+1)*16])
        curr_block = cipher.encrypt(curr_block)
        out += curr_block
    return out

def cbc_decrypt(ciphtext, IV, cipher):
    out = bytes([])
    curr_block = IV[:]
    for i in range(len(ciphtext) // 16):
        dec_block = cipher.decrypt(ciphtext[i*16:(i+1)*16])
        dec_block = xor(curr_block, dec_block)
        out += dec_block
        curr_block = ciphtext[i*16:(i+1)*16]
    return out

In [151]:
key = b'YELLOW SUBMARINE'
IV = key
cipher = AES.new(key, AES.MODE_ECB)

def enc1(msg):
    msg = msg.replace(';', '%3b').replace('=', '%3d')
    msg = "comment1=cooking%20MCs;userdata=" + msg + ";comment2=%20like%20a%20pound%20of%20bacon"
    try:
        msg.encode('ascii')
    except UnicodeDecodeError():
        raise Exception(msg)
    return cbc_encrypt(msg.encode('ascii'), IV, cipher)

def dec1(ciphtxt):
    msg = cbc_decrypt(ciphtxt, IV, cipher)
    try:
        msg.decode('ascii')
        return True
    except UnicodeDecodeError:
        raise Exception(msg.decode('ascii', 'replace'))

In [152]:
ciphtxt = enc1('attack at dawn, attack at dawn, attack at dawn!!')[:48]
try: 
    dec1(ciphtxt[:16] + bytes(16) + ciphtxt[:16])
except Exception as e:
    pptext = str(e).encode('ascii', "replace")
    rec_key = xor(pptext[:16], pptext[32:48]) 
    print(rec_key)

b'YELLOW SUBMARINE'


### Challenge 28

In [340]:
# based on pseudocode from https://en.wikipedia.org/wiki/SHA-1 

def chunks(lst, n):
    assert len(lst) % n == 0
    out = []
    for i in range(len(lst) // n):
        out.append(lst[i*n:(i+1)*n])
    return out
        
def leftrotate(x, n):
    out = (x << n) ^ (x >> (32 - n))
    return out

def md_pad(ml, custom_len=None):
    pad = bytes([0x80])
    plen = (448 - 8*(ml+1) % 512) // 8
    pad += bytes([0x00] * plen)
    pad += (8*custom_len if custom_len else 8*ml).to_bytes(8, 'big')
    return pad

def sha1(msg, h0=0x67452301, h1=0xEFCDAB89, h2=0x98BADCFE, h3=0x10325476, h4=0xC3D2E1F0, custom_len=None):
    msg += md_pad(len(msg), custom_len)
    assert len(msg) % 64 == 0
        
    for ch in range(0, len(msg), 64):      
        w = [int.from_bytes(b, 'big') for b in chunks(msg[ch:ch+64], 4)]
        assert len(w) == 16
    
        for i in range(16, 80):
            w.append(leftrotate(w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16], 1))
                        
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        
        for i in range(80):
            if i < 19:
                f = (b & c) | (~b & d)
                k = 0x5A827999
            elif i < 40:
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif i < 60:
                f = (b & c) | (b & d) | (c & d)
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6
                
            temp = (leftrotate(a, 5) + f + e + k + w[i]) % 2**32
            e = d
            d = c
            c = leftrotate(b, 30)
            b = a
            a = temp
            
        h0 = (h0 + a) % 2**32
        h1 = (h1 + b) % 2**32
        h2 = (h2 + c) % 2**32
        h3 = (h3 + d) % 2**32
        h4 = (h4 + e) % 2**32
                
    return (h0 << 128) | (h1 << 96) | (h2 << 64) | (h3 << 32) | h4

In [265]:
sha1(b'The quick brown fox jumps over the lazy dog').to_bytes(20, 'big').hex()

'0f9cb28ba119e4efcaf6477d7891351bc3105fd6'

In [266]:
sha1(b'The quick brown fox jumps over the hazy dog').to_bytes(20, 'big').hex()

'aa40657ab75bc665b02dbd7563579287a9303af5'

In [267]:
def auth(msg, mac, key):
    return mac == sha1(key+msg)
    
mac1 = sha1(b'key' + b'hello world')
auth(b'hello world', mac1, b'key')

True

### Challenge 29

In [268]:
def recover_state(h):
    return (h >> 128) & 0xffffffff, (h >> 96) & 0xffffffff, (h >> 64) & 0xffffffff, (h >> 32) & 0xffffffff, h & 0xffffffff

In [346]:
key = b'banana'
orig_msg = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
orig_hash = sha1(key+orig_msg)
append_str = b';admin=true'

h0, h1, h2, h3, h4 = recover_state(orig_hash)

for keylen in range(1, 17):
    forged_msg = key + orig_msg + md_pad(keylen + len(orig_msg)) + append_str
    new_hash = sha1(append_str, h0, h1, h2, h3, h4, len(forged_msg))
    if new_hash == sha1(forged_msg):
        print(keylen, new_hash)

6 414374220315846656680590240626294512667909987564


### Challenge 30

In [385]:
# based on description at https://tools.ietf.org/html/rfc1320

def F(X, Y, Z):
    return (X & Y) | (~X & Z)

def G(X, Y, Z):
    return (X & Y) | (X & Z) | (Y & Z)

def H(X, Y, Z):
    return X ^ Y ^ Z

def magic1(X, a, b, c, d, k, s):
    return leftrotate(a + F(b,c,d) + X[k], s)

def magic2(X, a, b, c, d, k, s):
    return leftrotate(a + G(b,c,d) + X[k] + 0x5a827999, s)

def magic3(X, a, b, c, d, k, s):
    return leftrotate(a + H(b,c,d) + X[k] + 0x6ed9eba1, s)


def md4(msg, A=0x01234567, B=0x89abcdef, C=0xfedcba98, D=0x76543210, custom_len=None):
    msg += md_pad(len(msg), custom_len)  # TODO: padding is a bit different

    for i in range(len(msg) // 64):  
        
        X = []
        for j in range(16):
            X.append(int.from_bytes(msg[4*(i*16+j):4*(i*16+j+1)], 'big'))
                                        
        AA = A
        BB = B
        CC = C
        DD = D
        
        A = magic1(X, A, B, C, D, 0, 3)
        A = magic1(X, A, B, C, D, 4, 3)
        A = magic1(X, A, B, C, D, 8, 3)
        A = magic1(X, A, B, C, D, 12, 3)
        D = magic1(X, D, A, B, C, 1, 7)
        D = magic1(X, D, A, B, C, 5, 7)
        D = magic1(X, D, A, B, C, 9, 7)
        D = magic1(X, D, A, B, C, 13, 7)
        C = magic1(X, C, D, A, B, 2, 11)
        C = magic1(X, C, D, A, B, 6, 11)
        C = magic1(X, C, D, A, B, 10, 11)
        C = magic1(X, C, D, A, B, 14, 11)
        B = magic1(X, B, C, D, A, 3, 19)
        B = magic1(X, B, C, D, A, 7, 19)
        B = magic1(X, B, C, D, A, 11, 19)
        B = magic1(X, B, C, D, A, 15, 19)
         
        A = magic2(X, A, B, C, D, 0, 3)
        A = magic2(X, A, B, C, D, 1, 3)
        A = magic2(X, A, B, C, D, 2, 3)
        A = magic2(X, A, B, C, D, 3, 3)
        D = magic2(X, D, A, B, C, 4, 5)
        D = magic2(X, D, A, B, C, 5, 5)
        D = magic2(X, D, A, B, C, 6, 5)
        D = magic2(X, C, A, B, C, 7, 5)
        C = magic2(X, C, D, A, B, 8, 9)
        C = magic2(X, C, D, A, B, 9, 9)
        C = magic2(X, C, D, A, B, 10, 9)
        C = magic2(X, C, D, A, B, 11, 9)
        B = magic2(X, B, C, D, A, 12, 13)
        B = magic2(X, B, C, D, A, 13, 13)
        B = magic2(X, B, C, D, A, 14, 13)
        B = magic2(X, B, C, D, A, 15, 13)
                
        A = magic3(X, A, B, C, D, 0, 3)
        A = magic3(X, A, B, C, D, 2, 3)
        A = magic3(X, A, B, C, D, 1, 3)
        A = magic3(X, A, B, C, D, 3, 3)
        D = magic3(X, D, A, B, C, 8, 9)
        D = magic3(X, D, A, B, C, 10, 9)
        D = magic3(X, D, A, B, C, 9, 9)
        D = magic3(X, D, A, B, C, 11, 9)
        C = magic3(X, C, D, A, B, 4, 11)
        C = magic3(X, C, D, A, B, 6, 11)
        C = magic3(X, C, D, A, B, 5, 11)
        C = magic3(X, C, D, A, B, 7, 11)
        B = magic3(X, B, C, D, A, 12, 15)
        B = magic3(X, B, C, D, A, 14, 15)
        B = magic3(X, B, C, D, A, 13, 15)
        B = magic3(X, B, C, D, A, 15, 15)
        
        A = (A + AA) % 2**32
        B = (B + BB) % 2**32
        C = (C + CC) % 2**32
        D = (D + DD) % 2**32
        
    return (A << 96) | (B << 64) | (C << 32) | D

In [386]:
md4(b'The quick brown fox jumps over the lazy dog').to_bytes(16, 'big').hex()

'3a19b5a64f9f6f6aba95ce31b23225ca'

In [387]:
md4(b'The quick brown fox jumps over the hazy dog').to_bytes(16, 'big').hex()

'4ed8851e8eed60810e72088419181558'

In [388]:
def recover_md_state(s):
    return (s >> 96) & 0xffffffff, (s >> 64) & 0xffffffff, (s >> 32) & 0xffffffff, s & 0xffffffff

In [389]:
key = b'banana'
orig_msg = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
orig_hash = md4(key+orig_msg)
append_str = b';admin=true'

A, B, C, D = recover_md_state(orig_hash)

for keylen in range(1, 17):
    forged_msg = key + orig_msg + md_pad(keylen + len(orig_msg)) + append_str
    new_hash = md4(append_str, A, B, C, D, len(forged_msg))
    if new_hash == md4(forged_msg):
        print(keylen, new_hash)

6 318318034757243947590078406317015093914


### Challenge 31
Sort of cheating here, since not using an actual web server.

In [409]:
def zero_pad(msg, block_size):
    return msg + bytes([0] * (block_size * math.ceil(len(msg) / block_size) - len(msg)))

In [435]:
# Based on pseudocode from https://en.wikipedia.org/wiki/HMAC

def hmac(key, msg, hash_fn, block_size, output_size):
    if len(key) > block_size:
        key = hash(key)
    if len(key) < block_size:
        key = zero_pad(key, block_size)
        
    o_key_pad = xor(key, bytes([0x5c] * block_size))
    i_key_pad = xor(key, bytes([0x36] * block_size))
    
    return hash_fn(o_key_pad + hash_fn(i_key_pad + msg).to_bytes(20, 'big'))

In [454]:
import time

key = random_bytes(8)

def process_request(msg, sig):
    return sig == hmac(key, msg, sha1, 64, 20)
    
def process_request_insecure(msg, sig, sleep_time):
    sig = sig.to_bytes(20, 'big')
    expected_sig = hmac(key, msg, sha1, 64, 20).to_bytes(20, 'big')
    for i in range(len(sig)):
        if sig[i] != expected_sig[i]:
            return False
        time.sleep(sleep_time)
    return True   

In [455]:
process_request(b'hello world', hmac(key, b'hello world', sha1, 64, 20))

True

In [456]:
process_request_insecure(b'hello world', hmac(key, b'hello world', sha1, 64, 20))

True

In [459]:
msg = b'hello world'
hash_guess = bytearray([0] * 20)

print(hmac(key, msg, sha1, 64, 20).to_bytes(20, 'big'))

for i in range(20):
    slowest_bit = 0
    longest_time = 0
    for j in range(256):
        hash_guess[i] = j
        t0 = time.time()
        # sleeping for as little as 5ms is OK for this approach
        process_request_insecure(msg, int.from_bytes(hash_guess, 'big'), 0.005)
        t1 = time.time()
        j_time = t1 - t0
        if j_time > longest_time:
            slowest_bit = j
            longest_time = j_time
    hash_guess[i] = slowest_bit
    print(bytes(hash_guess))

b'\xa7\x0b)\xff\xfd\x11\xaa#\x01\xfe7\xb2\xa6:enS\x85\x86~'
b'\xa7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\xaa\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\xaa#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\xaa#\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\xaa#\x01\xfe\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\xaa#\x01\xfe7\x00\x00\x00\x00\x00\x00\x00\x00\x00'
b'\xa7\x0b)\xff\xfd\x11\xaa#\x01\xfe7\xb2\x00\x00\x00\x00\

### Challenge 32

In [ ]:
msg = b'hello world'
hash_guess = bytearray([0] * 20)

print(hmac(key, msg, sha1, 64, 20).to_bytes(20, 'big'))

NUM_SAMPLES = 100
for i in range(20):
    slowest_bit = 0
    longest_time = 0
    for j in range(256):
        hash_guess[i] = j
        j_times = []
        for _ in range(NUM_SAMPLES):
            t0 = time.time()
            process_request_insecure(msg, int.from_bytes(hash_guess, 'big'), 0.001)
            t1 = time.time()
            j_times.append(t1 - t0)
        j_time = sum(j_times) / NUM_SAMPLES
        if j_time > longest_time:
            slowest_bit = j
            longest_time = j_time
    hash_guess[i] = slowest_bit
    print(bytes(hash_guess))